In [16]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import tensorflow as tf
from keras.callbacks import ModelCheckpoint,LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras.backend as K

cats=['alt.atheism','soc.religion.christian']
newsgroups_train = fetch_20newsgroups(subset='train',categories=cats)
newsgroups_test = fetch_20newsgroups(subset='test',categories=cats)
class_names = newsgroups_train.target_names
y_train = newsgroups_train.target
y_test = newsgroups_test.target

vectorizer = TfidfVectorizer(stop_words='english',lowercase=True,max_features=3000,max_df=10,ngram_range=(1,2),analyzer='word')
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_test = vectorizer.transform(newsgroups_test.data)
X_train=X_train.toarray()
X_test=X_test.toarray()

X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (1079, 3000)
X_test shape: (717, 3000)


In [17]:
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

from torch import Tensor

w1=[]
w2=[]
w3=[]

class data(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

    def __len__(self):
        return self.data.shape[0]

class my_model(nn.Module):
    def __init__(self):
        super(my_model, self).__init__()

        self.l1 = nn.Linear(3000, 1000)
        self.l2 = nn.Linear(1000, 500)
        self.l3 = nn.Linear(500, 250)
        self.l4 = nn.Linear(250, 1)

    def forward(self, x):
        x = F.relu(self.l1(x))
        w1.append(x.tolist())
        x = F.relu(self.l2(x))
        w2.append(x.tolist())
        x = F.relu(self.l3(x))
        w3.append(x.tolist())
        x = F.sigmoid(self.l4(x))
        return x.flatten()

In [18]:
train_data = data(X_train, y_train)
train_loader = DataLoader(train_data, 1, True)
model = my_model()
loss_fn = nn.BCELoss()
optim = Adam(model.parameters(), 0.001)
epochs = 1

for epoch in range(epochs):
    for i, dat in enumerate(train_loader):
        inp, labs = dat
        optim.zero_grad()
        outs = model(inp)
        loss = loss_fn(outs, labs)
        loss.backward()
        optim.step() 

c:\Users\91967\Anaconda3\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [19]:
len(w1[0][0]),len(w2[0][0]),len(w3[0][0])

(1000, 500, 250)

In [22]:
len(w1[0])

1

In [20]:
len(w1),len(w2),len(w3)

(1079, 1079, 1079)

In [27]:
layer1 = np.zeros((1000,1079))
for i in range(1000):
    for j in range(1079):
        layer1[i][j]=w1[j][0][i]

layer2 = np.zeros((500,1079))
for i in range(500):
    for j in range(1079):
        layer2[i][j]=w2[j][0][i]

layer3 = np.zeros((250,1079))
for i in range(250):
    for j in range(1079):
        layer3[i][j]=w3[j][0][i]